# ndvi_smol

### Here we gather Sentinel-2 data to calculate plot ndvi 

In [ ]:
# # For Sentinel-2
# # !pip install sentinelsat
# import sys
# !{sys.executable} -m pip install sentinelsat

In [ ]:
import ee
import geemap
import geopandas as gpd
from tqdm import tqdm
import pandas as pd
import os
# os.chdir('..')
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate
from scipy.stats import zscore
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings


pd.set_option('display.max_columns', None)

In [ ]:
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1))',
        {'NIR': image.select('B8'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('evi')
    ndwi = image.normalizedDifference(['B8', 'B11']).rename('ndwi')
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + 0.5)) * 1.5',
        {'NIR': image.select('B8'), 'RED': image.select('B4')}
    ).rename('savi')
    rendvi = image.normalizedDifference(['B8', 'B5']).rename('rendvi')
    mcari2 = image.expression(
        '((NIR - RE) - 0.2*(NIR - RED)) * (NIR / RE)',
        {'NIR': image.select('B8'), 'RED': image.select('B4'), 'RE': image.select('B5')}
    ).rename('mcari2')
    
    return image.addBands([ndvi, evi, ndwi, savi, rendvi, mcari2])



def mask_clouds(img, cloud_prob_threshold=20):
    '''Mask clouds using s2cloudless probability band.'''
    cloud_prob = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') \
        .filter(ee.Filter.eq('system:index', img.get('system:index'))).first()
    clouds = cloud_prob.gt(cloud_prob_threshold)
    mask = clouds.Not()
    return img.updateMask(mask)


def reduce_region(img, geom, plot_id):
    # all EE operations
    stats = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geom,
        scale=10,
        bestEffort=True,
        maxPixels=1e13
    )
    props = {k: stats.get(k) for k in [
        'ndvi',#'evi','ndwi','savi','rendvi','mcari2'
    ]}
    props['date'] = img.date().format('YYYY-MM-dd')
    props['plot_id'] = plot_id
    return ee.Feature(None, props)



## Here we initialize our access to the satellite data download. 
## Requires authentication via browser.

In [ ]:
ee.Authenticate()
ee.Initialize()

# load the vineyard polygon (GeoJSON)
vineyard = pd.read_pickle('../data/polygons/RegressionRidge_smol_smol.pkl')


from shapely.geometry import Polygon, MultiPolygon

# your existing GeoDataFrame
gdf = vineyard.copy()

rows = []
for idx, row in gdf.iterrows():
    geom = row.geometry
    parent = row['plot_id']
    
    if isinstance(geom, Polygon):
        rows.append({"geometry": geom, "parent_plot": parent})
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            rows.append({"geometry": poly, "parent_plot": parent})

# create new flattened GeoDataFrame
vineyard_flat = gpd.GeoDataFrame(rows, crs=gdf.crs)



geoms = [ee.Geometry.Polygon(list(p.exterior.coords)) for p in vineyard_flat.geometry]


In [ ]:
gdf

### The below code will loop over a given set of years starting in 2016 (earliest available data), search for the polygon coordinates and compute ndvi for our plots. Finally, the data is serialized as a pandas pickle file.

Set dates to search over. Make sure there is a 'plots' folder for the plot ndvi data

In [ ]:
years = [str(year) for year in range(2016, 2025)]
months = ['01', '12']
days = ['01', '31']
start_dates = [f'{year}-{months[0]}-{days[0]}' for year in years]
end_dates   = [f'{year}-{months[1]}-{days[1]}' for year in years]

os.makedirs('data/ndvi/plots', exist_ok=True)

Loop over years and search database for images overlapping our polygons. Then download the and st

In [ ]:
# --- Flatten MultiPolygons if necessary ---
def shapely_to_ee_feature(poly, plot_id):
    """Convert a Polygon or MultiPolygon to EE Feature with plot_id property."""
    if poly.geom_type == 'Polygon':
        return ee.Feature(ee.Geometry.Polygon(list(poly.exterior.coords)), {'plot_id': plot_id})
    elif poly.geom_type == 'MultiPolygon':
        features = []
        for p in poly.geoms:
            features.append(ee.Feature(ee.Geometry.Polygon(list(p.exterior.coords)), {'plot_id': plot_id}))
        return ee.FeatureCollection(features)
    else:
        raise ValueError(f"Unsupported geometry type: {poly.geom_type}")

In [ ]:
# Flatten GeoDataFrame into FeatureCollection
ee_features = []
for idx, row in tqdm(vineyard_flat.iterrows()):
    poly = row.geometry
    parent = row.get('parent_plot', f'plot_{idx}')
    feat = shapely_to_ee_feature(poly, parent)
    
    if isinstance(feat, ee.FeatureCollection):
        ee_features.extend(feat.toList(feat.size()).getInfo())
    else:
        ee_features.append(feat.getInfo())
        
# hex_fc['geometry'] = hex_fc.geometry.buffer(5)  # buffer by 5 meters

hex_fc = ee.FeatureCollection(ee_features)

In [ ]:
# hex_fc_ee = hex_fc.map(lambda f: f.setGeometry(f.geometry().transform('EPSG:4326')))


In [ ]:
for start_date, end_date in zip(start_dates, end_dates):
    break
    '''
    Undo the break statement to download the data
    '''
    
    # fname = f'../data/ndvi/plots/ndvi_{start_date}_to_{end_date}.pkl'
    # if os.path.isfile(fname):
    #     continue

    # Load Sentinel-2 collection and add indices
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(hex_fc)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .select(['B2', 'B4', 'B5', 'B8', 'B11'])
        .map(add_indices)
    )

    # Server-side extraction: map reduce_region over each hex for each image
    def extract_features(img):
        return hex_fc.map(
            lambda f: reduce_region(
                img, 
                f.geometry(), 
                f.get('plot_id')
            )
        )

    # Flatten all features across all images
    fc = collection.map(extract_features).flatten()

    # Export to Google Drive as CSV to avoid memory issues
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=f'ndvi_{start_date}_to_{end_date}',
        folder=f'ee_ndvi_exports_{start_date}_to_{end_date}',
        fileFormat='CSV'
    )
    task.start()
    print(f"Export started for {start_date} to {end_date}. Check Google Drive folder 'ee_ndvi_exports'.")


In [ ]:
'''
code to check on earth engine task status
'''
# import ee
# ee.Initialize()

# tasks = ee.batch.Task.list()

# for t in tasks:
#     print(f"ID: {t.id}")
#     print(f"Type: {t.task_type}")
#     print(f"Description: {t.config.get('description')}")
#     print(f"State: {t.state}\n")


In [ ]:
ndvi_files = [
    'ndvi_2016-01-01_to_2016-12-31.csv',
    'ndvi_2017-01-01_to_2017-12-31.csv',
    'ndvi_2018-01-01_to_2018-12-31.csv',
    'ndvi_2019-01-01_to_2019-12-31.csv',
    'ndvi_2020-01-01_to_2020-12-31.csv',
    'ndvi_2021-01-01_to_2021-12-31.csv',
    'ndvi_2022-01-01_to_2022-12-31.csv',
    'ndvi_2023-01-01_to_2023-12-31.csv',
    'ndvi_2024-01-01_to_2024-12-31.csv',
    
]

In [ ]:

dfs = []
data_path = '../data/ndvi/plots/'

for file_path in tqdm(ndvi_files):

    ndvi_path = os.path.join(data_path, file_path)
    df = pd.read_csv(ndvi_path)
    df = df[['date','ndvi', 'plot_id']]
    
    dfs.append(df)
    
df = pd.concat(dfs, axis = 0)
df = df.reset_index(drop = True)

In [ ]:
df

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def hampel_filter_series(s, window=7, n_sigmas=3):
    """
    Hampel filter for 1D pandas Series.
    
    Parameters:
    - s: pd.Series, time-ordered
    - window: half-window size (total window = 2*window+1)
    - n_sigmas: threshold multiplier
    
    Returns:
    - pd.Series with outliers replaced by NaN
    """
    k = 1.4826  # scale factor for MAD
    s_clean = s.copy()

    # rolling median and MAD
    rolling_med = s.rolling(window=2*window+1, center=True, min_periods=1).median()
    rolling_mad = s.rolling(window=2*window+1, center=True, min_periods=1).apply(
        lambda x: np.median(np.abs(x - np.median(x))), raw=True
    )

    # avoid zero MAD
    rolling_mad[rolling_mad == 0] = 1e-6

    # compute deviations
    diff = np.abs(s - rolling_med)
    threshold = n_sigmas * k * rolling_mad

    # replace outliers with NaN
    s_clean[diff > threshold] = np.nan
    return s_clean

# list of columns to filter
cols_to_filter = [
    'ndvi', #'evi', 'ndwi', 'savi', 'rendvi', 'mcari2'
]

for col in tqdm(cols_to_filter):
    # apply Hampel filter group-wise
    df[col] = df.groupby('plot_id')[col].transform(
        lambda s: hampel_filter_series(s, window=7, n_sigmas=2)
    )
    
    # smooth filtered series
    df[f'{col}_smooth'] = df.groupby('plot_id')[col].transform(
        lambda s: s.rolling(window=7, center=True, min_periods=1).mean()
    )

print("Hampel filter and smoothing done!")


In [ ]:
df

### Let's do a quick stats check here. Figure out how confident we are in these smoothed curves

In [ ]:
# for col in ['ndvi', 'evi', 'ndwi', 'savi', 'rendvi', 'mcari2']:
#     df[col + '_residual'] = df[col] - df[col + '_smooth']
#     df[col + '_residual_pct'] = (df[col] - df[col + '_smooth']) / df[col + '_smooth'] * 100
    

In [ ]:
df.isna().sum()

In [ ]:
import numpy as np

# Identify your index columns
id_cols = ["plot_id", "date"]

# Raw indices
raw_cols = [
    "ndvi",#"evi","ndwi","savi","rendvi","mcari2"
]

# Smooth versions
smooth_cols = ['ndvi_smooth']#[c+"_smooth" for c in raw_cols]

# Step 1: Fill raw with smoothed, then median per plot, then global median
for raw, smooth in zip(raw_cols, smooth_cols):
    
    df[raw] = df[raw].fillna(df[smooth])  # use smoothed
    df[raw] = df.groupby("plot_id")[raw].transform(lambda x: x.fillna(x.median()))
    df[raw] = df[raw].fillna(df[raw].median())

# Step 2: Fill smooth (only a few NaNs)
for col in smooth_cols:
    df[col] = df.groupby("plot_id")[col].transform(lambda x: x.fillna(x.median()))
    df[col] = df[col].fillna(df[col].median())

# Step 3: Recompute residuals
for raw, smooth in zip(raw_cols, smooth_cols):
    resid = raw + "_residual"
    resid_pct = raw + "_residual_pct"
    df[resid] = df[raw] - df[smooth]
    # To avoid divide-by-zero, use np.where
    df[resid_pct] = np.where(df[smooth] != 0, df[resid] / df[smooth], np.nan)


In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
plot_id = 23    # example plot
year = 2024    # example year

df_plot = df[(df['plot_id']==plot_id) & (df['date'].dt.year==year)].sort_values('date')

indices = [
    'ndvi',#'evi','ndwi','savi','rendvi','mcari2'
]

# plt.figure(figsize=(14,6))
for idx in indices:
    plt.plot(df_plot['date'], df_plot[idx], 'o-', alpha=0.5, label=f'{idx} raw')
    sm = idx + '_smooth'
    if sm in df_plot.columns:
        plt.plot(df_plot['date'], df_plot[sm], '-', lw=2, label=f'{idx} smooth')
    plt.title(f'Plot {plot_id} - {year} index time series')
    plt.xlabel('Date')
    plt.ylabel('Index value')
    plt.legend()
    plt.show()


In [ ]:
veg_features = [
    'ndvi_smooth',
                # 'evi_smooth', 'ndwi_smooth', 'savi_smooth', 'rendvi_smooth', 'mcari2_smooth'
]

df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.week

In [ ]:
df

In [ ]:
df.to_pickle('ndvi_df_smoothed.pkl')

In [ ]:
df = pd.read_pickle('ndvi_df_smoothed.pkl')

In [ ]:
df_orig = df.copy()
df = df.copy()

In [ ]:
df = df_orig.copy()

In [ ]:
df = df[
    (df['week'] > 27) &
    (df['week'] < 46)
].copy()

In [ ]:
def stretch_to_daily_per_year(g):
    # g is all data for one plot_id
    daily_parts = []
    for year, g_year in (g.groupby(g['date'].dt.year)):
        g_year = g_year.copy()
        numeric_cols = g_year.select_dtypes(include=[np.number]).columns.tolist()
        if 'date' in numeric_cols:
            numeric_cols.remove('date')
        
        # Collapse duplicate dates
        g_year = (g_year.groupby('date')
                  .agg({**{c: 'mean' for c in numeric_cols},
                        **{c: 'first' for c in g_year.columns.difference(numeric_cols + ['date'])}})
                  .reset_index()
                 )
        
        g_year = g_year.set_index('date').sort_index()
        full_idx = pd.date_range(start=g_year.index.min(), end=g_year.index.max(), freq='D')
        g_year = g_year.reindex(full_idx)
        
        g_year['plot_id'] = g_year['plot_id'].ffill().bfill()
        for col in numeric_cols:
            g_year[col] = g_year[col].interpolate(method='time', limit_direction='both')
        
        non_numeric = [c for c in g_year.columns if c not in numeric_cols + ['plot_id']]
        if non_numeric:
            g_year[non_numeric] = g_year[non_numeric].ffill().bfill()
        
        g_year = g_year.reset_index().rename(columns={'index':'date'})
        daily_parts.append(g_year)
    
    return pd.concat(daily_parts, axis=0)


In [ ]:
full_daily = df.groupby('plot_id', as_index=False).apply(lambda g: stretch_to_daily_per_year(g)).reset_index(drop=True)


In [ ]:
full_daily['week'] = full_daily['date'].dt.week

full_daily['year'] = full_daily['date'].dt.year

full_daily.sort_values('date')

In [ ]:
weekly_stats_list = []

for (plot, year, week), group in tqdm(full_daily.groupby([full_daily['plot_id'], full_daily['year'], full_daily['week']])):
    # print(plot, year,iiii week, group)
    week_dict = {'plot_id': plot, 'year': year, 'week': week}
    
    # convert dates to numeric ordinals for slope calculation
    x = group['date'].map(pd.Timestamp.toordinal).values
    
    for feat in veg_features:
        y = group[feat].values
        
        # compute slope (direction matters)
        week_dict[f'{feat}_slope'] = np.polyfit(x, y, 1)[0] if len(x) > 1 else np.nan
        
        # compute weekly summary statistics
        week_dict[f'{feat}_mean'] = np.mean(y)
        week_dict[f'{feat}_std']  = np.std(y)
        week_dict[f'{feat}_max']  = np.max(y)
        week_dict[f'{feat}_min']  = np.min(y)
    
    weekly_stats_list.append(week_dict)
    # print(weekly_stats_list)
    # break

weekly_stats_df = pd.DataFrame(weekly_stats_list)


In [ ]:
wsdf = weekly_stats_df.copy()
# wsdf = wsdf[wsdf['week'].isin([(i) for i in range(28, 45)])]

In [ ]:
veg_agg = wsdf.copy() 

In [ ]:
drop_cols = [col for col in veg_agg.columns if 'max' in col or 'min' in col]

In [ ]:
veg_agg = veg_agg.drop(columns = drop_cols)

In [ ]:
slope_cols = [c for c in veg_agg.columns if c.endswith('_slope')]
std_cols = [c for c in veg_agg.columns if c.endswith('_std')]

veg_agg[slope_cols] = veg_agg[slope_cols].fillna(0)
veg_agg[std_cols] = veg_agg[std_cols].fillna(0)


In [ ]:
# Pivot so that each month becomes a set of columns
veg_agg_wide = veg_agg.pivot_table(
    index=['plot_id','year'],
    columns='week',
    values=[col for col in veg_agg.columns if col not in ['plot_id','year','week']]
).reset_index()

# Flatten multi-index columns
veg_agg_wide.columns = ['_'.join([str(c) for c in col if c != '']) for col in veg_agg_wide.columns]


In [ ]:
ndvi_cols = [col for col in veg_agg_wide.columns if col.startswith('ndvi_smooth_mean')]

veg_agg_wide['ndvi_cov'] = veg_agg_wide[ndvi_cols].std(axis=1) / veg_agg_wide[ndvi_cols].mean(axis=1)
veg_agg_wide['ndvi_mean'] = veg_agg_wide[ndvi_cols].mean(axis = 1)
veg_agg_wide['ndvi_std'] = veg_agg_wide[ndvi_cols].std(axis = 1)

In [ ]:
veg_agg_wide.to_pickle('../data/ndvi/plots/final_df.pkl')